<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/symbolic_theory_end_to_end_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# symbolic_theory_end_to_end.py
from __future__ import annotations

import os
import re
from dataclasses import dataclass
from itertools import combinations
from typing import Any, Dict, List, Tuple, Optional

import sympy as sp

try:
    import networkx as nx  # optional
    _HAS_NX = True
except Exception:
    _HAS_NX = False


# ------------------------- Utilities -------------------------

def _sanitize_name(x: Any) -> str:
    s = str(x)
    s = re.sub(r"[^0-9a-zA-Z_]", "_", s)
    if not s or s[0].isdigit():
        s = f"n_{s}"
    return s


def _graph_nodes_and_edges(g: Any) -> Tuple[List[Any], List[Tuple[Any, Any]]]:
    # networkx
    if _HAS_NX and isinstance(g, (nx.Graph, nx.DiGraph)):
        nodes = list(g.nodes())
        edges = list(g.edges())
        return nodes, edges

    # causal-learn-like Graph
    if hasattr(g, "node_num") and hasattr(g, "is_directed"):
        nodes = list(range(getattr(g, "node_num")))
        edges = []
        for i in nodes:
            for j in nodes:
                if i == j:
                    continue
                try:
                    if g.is_directed(i, j):
                        edges.append((i, j))
                except Exception:
                    pass
        edges = list(dict.fromkeys(edges))
        return nodes, edges

    # Generic
    if hasattr(g, "nodes") and callable(getattr(g, "nodes")):
        nodes = list(g.nodes())
    elif hasattr(g, "nodes"):
        nodes = list(g.nodes)
    else:
        raise TypeError("Unsupported graph type: cannot extract nodes.")

    if hasattr(g, "edges") and callable(getattr(g, "edges")):
        edges = list(g.edges())
    elif hasattr(g, "edges"):
        edges = list(g.edges)
    else:
        edges = []
    return nodes, edges


# ------------------------- Configuration -------------------------

@dataclass
class BuildConfig:
    restrict_to_edges: bool = False
    add_constant: bool = True
    var_prefix: str = "v"


@dataclass
class SolveConfig:
    # Try to solve by setting one variable (anchor) and solving others
    anchor_value: float = 1.0
    max_equations: int = 20  # limit solve complexity
    simplify: bool = True


# ------------------------- Constructor -------------------------

class SymbolicTheoryConstructor:
    def __init__(self, var_prefix: str = "v"):
        self.var_prefix = var_prefix

    def _build_var_map(self, nodes: List[Any]) -> Dict[Any, sp.Symbol]:
        return {n: sp.Symbol(f"{self.var_prefix}_{_sanitize_name(n)}", real=True) for n in nodes}

    def build(self, causal_graph: Any, cfg: BuildConfig | None = None) -> Dict[str, Any]:
        if cfg is None:
            cfg = BuildConfig(var_prefix=self.var_prefix)
        else:
            if cfg.var_prefix != self.var_prefix:
                cfg = BuildConfig(
                    restrict_to_edges=cfg.restrict_to_edges,
                    add_constant=cfg.add_constant,
                    var_prefix=self.var_prefix,
                )

        nodes, edges = _graph_nodes_and_edges(causal_graph)
        if not nodes:
            raise ValueError("Graph has no nodes.")
        var_map = self._build_var_map(nodes)

        # choose pairs
        if cfg.restrict_to_edges and edges:
            undirected_pairs = {tuple(sorted((u, v))) for (u, v) in edges}
            pairs = sorted(list(undirected_pairs))
        else:
            pairs = list(combinations(nodes, 2))

        eqs: List[sp.Equality] = []
        for u, v in pairs:
            u_name = _sanitize_name(u)
            v_name = _sanitize_name(v)
            a, b = sp.symbols(f"a_{u_name}_{v_name} b_{u_name}_{v_name}", real=True)
            expr = a * var_map[u] + b * var_map[v]
            if cfg.add_constant:
                c = sp.symbols(f"c_{u_name}_{v_name}", real=True)
                expr = expr + c
            eqs.append(sp.Eq(expr, 0))

        theory = {"equations": eqs, "var_map": var_map, "graph": causal_graph, "config": cfg}
        return theory

    # ---------- Utilities ----------
    def pretty_print(self, theory: Dict[str, Any], latex: bool = False) -> str:
        eqs = theory.get("equations", [])
        return "\n".join(sp.latex(eq) if latex else str(eq) for eq in eqs)

    def with_sample_values(self, theory: Dict[str, Any], coeff_value: float = 1.0, const_value: float = 0.0) -> List[sp.Equality]:
        subs = {}
        for eq in theory["equations"]:
            for sym in eq.free_symbols:
                s = str(sym)
                if s.startswith(("a_", "b_")):
                    subs[sym] = coeff_value
                elif s.startswith("c_"):
                    subs[sym] = const_value
        return [sp.Eq(sp.simplify(eq.lhs.subs(subs)), 0) for eq in theory["equations"]]

    def deduplicate_equations(self, eqs: List[sp.Equality]) -> List[sp.Equality]:
        """
        Remove duplicates and trivial identities (0 = 0) after normalization.
        """
        canon = []
        seen = set()
        for eq in eqs:
            lhs = sp.simplify(eq.lhs)
            if lhs == 0:
                # equation like 0 = 0 (degenerate)
                continue
            # Normalize by dividing by the first nonzero coefficient to avoid scalar multiples
            coeffs = list(lhs.as_coefficients_dict().values())
            scale = None
            for c in coeffs:
                if c != 0:
                    scale = c
                    break
            if scale not in (None, 0):
                lhs_norm = sp.simplify(lhs / scale)
            else:
                lhs_norm = lhs
            key = sp.srepr(lhs_norm)
            if key not in seen:
                seen.add(key)
                canon.append(sp.Eq(lhs_norm, 0))
        return canon

    def try_solve(self, theory: Dict[str, Any], solve_cfg: SolveConfig | None = None) -> Dict[str, Any]:
        """
        Attempt to solve a subset of equations by anchoring one variable
        and solving for the rest symbolically.
        """
        if solve_cfg is None:
            solve_cfg = SolveConfig()

        eqs = list(theory["equations"])[: solve_cfg.max_equations]
        if not eqs:
            return {"status": "no_equations", "solution": None}

        # pick an anchor variable (first in var_map)
        var_map: Dict[Any, sp.Symbol] = theory["var_map"]
        vars_list = list(var_map.values())
        if not vars_list:
            return {"status": "no_variables", "solution": None}

        anchor = vars_list[0]
        subs = {anchor: solve_cfg.anchor_value}

        # eliminate coefficient symbols by substituting sample values (to avoid underdetermination)
        eqs_sampled = self.with_sample_values({"equations": eqs}, coeff_value=1.0, const_value=0.0)

        # apply anchor substitution
        eqs_sub = [sp.Eq(sp.simplify(eq.lhs.subs(subs)), 0) for eq in eqs_sampled]

        # deduplicate
        eqs_canon = self.deduplicate_equations(eqs_sub)

        # solve for remaining variables (best effort)
        unknowns = [v for v in vars_list if v != anchor]
        try:
            sol = sp.solve(eqs_canon, unknowns, dict=True)
            status = "ok" if sol else "underdetermined_or_inconsistent"
        except Exception as e:
            sol = [{"error": str(e)}]
            status = "error"

        return {
            "status": status,
            "anchor": str(anchor),
            "anchor_value": solve_cfg.anchor_value,
            "equations_used": len(eqs_canon),
            "solution": sol,
        }

    def export_summary(self, theory: Dict[str, Any], solve_result: Dict[str, Any]) -> str:
        """
        Produce a changelog-friendly summary of the theory and solve attempt.
        """
        cfg: BuildConfig = theory["config"]
        vm = theory["var_map"]
        eqs = theory["equations"]

        lines = []
        lines.append("=== Theory Summary ===")
        lines.append(f"- Mode: {'edges-only' if cfg.restrict_to_edges else 'all-pairs'}; constant term: {cfg.add_constant}")
        lines.append(f"- Variables: {', '.join(str(s) for s in vm.values())}")
        lines.append(f"- Equations: {len(eqs)}")
        lines.append("")
        lines.append("Sample equations (up to 5):")
        for eq in eqs[:5]:
            lines.append(f"  • {eq}")

        lines.append("")
        lines.append("=== Solve Attempt ===")
        lines.append(f"- Status: {solve_result.get('status')}")
        if "anchor" in solve_result:
            lines.append(f"- Anchor: {solve_result['anchor']} = {solve_result.get('anchor_value')}")
        lines.append(f"- Equations used: {solve_result.get('equations_used', 0)}")
        sol = solve_result.get("solution")
        if isinstance(sol, list) and sol and isinstance(sol[0], dict):
            lines.append("- Solution (first assignment):")
            first = sol[0]
            for k, v in first.items():
                lines.append(f"  {k} = {v}")
        elif sol:
            lines.append(f"- Solution: {sol}")
        else:
            lines.append("- Solution: none")

        return "\n".join(lines)


# ------------------------- Demo / Main -------------------------

def _build_demo_graph():
    if _HAS_NX:
        G = nx.DiGraph()
        G.add_nodes_from(["x", "y", "z"])
        G.add_edges_from([("x", "y"), ("y", "z")])
        return G
    # Fallback
    class _G:
        def __init__(self):
            self._nodes = ["x", "y", "z"]
            self._edges = [("x", "y"), ("y", "z")]
        def nodes(self): return self._nodes
        def edges(self): return self._edges
    return _G()

def main():
    print("=== Symbolic Theory: Build • Solve • Export ===")

    # Build theories (edges-only and all-pairs)
    G = _build_demo_graph()
    ctor = SymbolicTheoryConstructor(var_prefix="v")

    cfg_edges = BuildConfig(restrict_to_edges=True, add_constant=True, var_prefix="v")
    theory_edges = ctor.build(G, cfg_edges)
    solve_edges = ctor.try_solve(theory_edges, SolveConfig(anchor_value=1.0, max_equations=10))
    summary_edges = ctor.export_summary(theory_edges, solve_edges)

    cfg_all = BuildConfig(restrict_to_edges=False, add_constant=True, var_prefix="v")
    theory_all = ctor.build(G, cfg_all)
    solve_all = ctor.try_solve(theory_all, SolveConfig(anchor_value=1.0, max_equations=10))
    summary_all = ctor.export_summary(theory_all, solve_all)

    # Print human view
    print("\n--- Edges-only ---")
    print(ctor.pretty_print(theory_edges))
    print("\n" + summary_edges)

    print("\n--- All-pairs ---")
    print(ctor.pretty_print(theory_all))
    print("\n" + summary_all)

    # Optional: write summaries to files
    out_dir = "theory_reports"
    os.makedirs(out_dir, exist_ok=True)
    with open(os.path.join(out_dir, "edges_only.txt"), "w", encoding="utf-8") as f:
        f.write(summary_edges)
    with open(os.path.join(out_dir, "all_pairs.txt"), "w", encoding="utf-8") as f:
        f.write(summary_all)
    with open(os.path.join(out_dir, "edges_only.tex"), "w", encoding="utf-8") as f:
        f.write(ctor.pretty_print(theory_edges, latex=True))
    with open(os.path.join(out_dir, "all_pairs.tex"), "w", encoding="utf-8") as f:
        f.write(ctor.pretty_print(theory_all, latex=True))

    print(f"\n[Saved] Reports to {out_dir}/ (txt and tex)")

if __name__ == "__main__":
    main()